[NWB (Neurodata Without Borders)](https://www.nwb.org/) is a data format proposed to become a data standard for neurophysiology.


In [50]:
from pathlib import Path

import numpy as np
import pynwb
from dandi.download import download as dandi_download
from nwbwidgets import nwb2widget


DATA_DIR = Path("../data/nwb")

Let's download data.

In [2]:
if not DATA_DIR.exists():
    DATA_DIR.mkdir(parents=True)
    dandi_download("DANDI:000207/0.220216.0323", output_dir=DATA_DIR)

PATH                                          SIZE     DONE            DONE% CHECKSUM STATUS          MESSAGE   
000207/dandiset.yaml                                                                  done            updated   
000207/sub-16/sub-16_ses-16_ecephys+image.nwb 2.7 MB   2.7 MB           100%    ok    done                      
000207/sub-11/sub-11_ses-11_ecephys+image.nwb 1.4 MB   1.4 MB           100%    ok    done                      
000207/sub-1/sub-1_ses-1_ecephys+image.nwb    5.2 MB   5.2 MB           100%    ok    done                      
000207/sub-10/sub-10_ses-10_ecephys+image.nwb 1.5 MB   1.5 MB           100%    ok    done                      
000207/sub-18/sub-18_ses-18_ecephys+image.nwb 3.5 MB   3.5 MB           100%    ok    done                      
000207/sub-13/sub-13_ses-13_ecephys+image.nwb 2.2 MB   2.2 MB           100%    ok    done                      
000207/sub-14/sub-14_ses-14_ecephys+image.nwb 2.9 MB   2.9 MB           100%    ok    done      

It looks like that the data is downloaded under `000207` directory in the data directory I specified. Let's prepare a convenient function for reading the data.

In [4]:
def load_nwb(session_id: int) -> pynwb.NWBFile:
    """Load an NWB file. session_id should be an integer from 1 to 19."""
    path = DATA_DIR.joinpath(
        f"000207/sub-{session_id}/sub-{session_id}_ses-{session_id}_ecephys+image.nwb"
    )
    return pynwb.NWBHDF5IO(path, mode="r").read()

While I already roughly read the paper, I have no idea what these files contain. 
To help me check the data content, NWB provides a separated package called [nwbwidgets](https://nwb-overview.readthedocs.io/en/latest/tools/nwbwidgets/nwbwidgets.html) that creates an interactive [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/stable/).
Let's load a data and visualize it.

In [56]:
nwb10 = load_nwb(10)
nwb2widget(nwb10)

/home/yuji/Documents/rlog2/.venv/lib/python3.11/site-packages/pynwb/core.py:47: UserWarning: OpticalSeries 'StimulusPresentation_encoding': The number of frame indices in 'starting_frame' should have the same length as 'external_file'.
  warn(error_msg)
/home/yuji/Documents/rlog2/.venv/lib/python3.11/site-packages/pynwb/core.py:47: UserWarning: OpticalSeries 'StimulusPresentation_encoding': Either external_file or data must be specified (not None), but not both.
  warn(error_msg)


It didn't helped me understand the data much..., but at least, I now know that `units/Session Raster` shows me a raster plot of the spiking data.
So I guess that the actual spiking data is under `units`.

In [5]:
nwb10 = load_nwb(10)

/home/yuji/Documents/rlog2/.venv/lib/python3.11/site-packages/pynwb/core.py:47: UserWarning: OpticalSeries 'StimulusPresentation_encoding': The number of frame indices in 'starting_frame' should have the same length as 'external_file'.
  warn(error_msg)
/home/yuji/Documents/rlog2/.venv/lib/python3.11/site-packages/pynwb/core.py:47: UserWarning: OpticalSeries 'StimulusPresentation_encoding': Either external_file or data must be specified (not None), but not both.
  warn(error_msg)


In [10]:
nwb10.stimulus

{'StimulusPresentation_encoding': StimulusPresentation_encoding pynwb.image.OpticalSeries at 0x140040236018640
 Fields:
   comments: no comments
   conversion: 1.0
   data: <HDF5 dataset "data": shape (2, 2, 2), type "<f8">
   description: no description
   dimension: <HDF5 dataset "dimension": shape (3,), type "<i4">
   distance: 0.7
   external_file: <HDF5 dataset "external_file": shape (90,), type "|O">
   field_of_view: <HDF5 dataset "field_of_view": shape (3,), type "<f8">
   format: external
   interval: 1
   offset: 0.0
   orientation: lower left
   resolution: -1.0
   starting_frame: [0]
   timestamps: <HDF5 dataset "timestamps": shape (90,), type "<f8">
   timestamps_unit: seconds
   unit: meters}

In [14]:
nwb10.stimulus["StimulusPresentation_encoding"].data[:]

array([[[nan, nan],
        [nan, nan]],

       [[nan, nan],
        [nan, nan]]])

In [20]:
nwb10.electrode_groups["NLX-microwires-137"]

NLX-microwires-137 pynwb.ecephys.ElectrodeGroup at 0x140040256734864
Fields:
  description: Microwire
  device: NLX-microwires-137 pynwb.device.Device at 0x140040248428240
Fields:
  description: Recordings were performed with Macro-Micro Hybrid Depth Electrodes with Behnke Fried/Micro Inner Wire Bundle in which each individual microwire has a diameter of 40 microns. Likwise, each Depth Electrode has 8 microwires.

  location: hippocampus_left

In [23]:
nwb10.acquisition["events"]

events pynwb.base.TimeSeries at 0x140040248427088
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (1720,), type "|i1">
  description: The events coorespond to the TTL markers for each trial. For the encoding trials, the TTL markers are the following: 61 = start of the experiment, 1 = stimulus ON, 2 = stimulus OFF, 11 = Fix Cross, 3 = Probe, 4 = Response, 60 = End of Experiment
  interval: 1
  offset: 0.0
  resolution: -1.0
  timestamps: <HDF5 dataset "timestamps": shape (1720,), type "<f8">
  timestamps_unit: seconds
  unit: NA

In [36]:
nwb10.units.spike_times.shape

(140083,)

In [45]:
nwb10.intervals["encoding_table"]

encoding_table pynwb.epoch.TimeIntervals at 0x140040235456336
Fields:
  colnames: ['start_time' 'stop_time' 'fixcross_time' 'ExperimentID' 'boundary1_time'
 'boundary2_time' 'boundary3_time' 'stimCategory' 'Clip_name']
  columns: (
    start_time <class 'hdmf.common.table.VectorData'>,
    stop_time <class 'hdmf.common.table.VectorData'>,
    fixcross_time <class 'hdmf.common.table.VectorData'>,
    ExperimentID <class 'hdmf.common.table.VectorData'>,
    boundary1_time <class 'hdmf.common.table.VectorData'>,
    boundary2_time <class 'hdmf.common.table.VectorData'>,
    boundary3_time <class 'hdmf.common.table.VectorData'>,
    stimCategory <class 'hdmf.common.table.VectorData'>,
    Clip_name <class 'hdmf.common.table.VectorData'>
  )
  description: intervals for the encoding task
  id: id <class 'hdmf.common.table.ElementIdentifiers'>

In [53]:
nwb10.intervals["encoding_table"].start_time[:]

array([  1.1404295 ,  10.3158655 ,  19.339164  ,  28.46748775,
        37.1526995 ,  45.7848485 ,  63.089899  ,  72.43824325,
        81.59445025,  90.73273125, 100.12601775, 109.18814375,
       117.742145  , 130.40028475, 139.3834105 , 148.41635425,
       157.401228  , 166.6578065 , 175.64558475, 190.36648875,
       199.416104  , 208.399781  , 217.6347525 , 226.47758275,
       240.78742875, 249.87464425, 259.25654825, 268.48181725,
       277.56536225, 286.59772075, 297.8551905 , 306.9603105 ,
       316.18439125, 324.778364  , 334.02651475, 343.16366225,
       351.05137575, 361.909297  , 370.9656565 , 379.90795275,
       388.7606235 , 397.03696475, 415.75064375, 425.3681415 ,
       434.79651175, 443.99076425, 453.228418  , 467.71653775,
       476.6149805 , 485.5766605 , 494.54777925, 503.621363  ,
       511.8541325 , 521.16817025, 536.9280785 , 546.0343685 ,
       554.89148425, 564.18862   , 573.25371625, 582.3744625 ,
       591.3386365 , 605.2981825 , 614.2894705 , 623.66

In [54]:
nwb10.units

units pynwb.misc.Units at 0x140040234099600
Fields:
  colnames: ['spike_times' 'electrodes']
  columns: (
    spike_times_index <class 'hdmf.common.table.VectorIndex'>,
    spike_times <class 'hdmf.common.table.VectorData'>,
    electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  )
  description: units table
  id: id <class 'hdmf.common.table.ElementIdentifiers'>
  waveform_unit: volts